# Importing Data

In [1]:
# Install using "pip install surprise"
import numpy as np
import numba
import matplotlib.pyplot as plt

In [13]:
# Matrix factorization functions from homework

@numba.jit
def grad_U(Ui, Yij, Vj, reg, eta):
    """
    Takes as input Ui (the ith row of U), a training point Yij, the column
    vector Vj (jth column of V^T), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Ui multiplied by eta.
    """
    # Gradient with respect to Ui
    return eta*(reg*Ui-Vj*(Yij-np.dot(Ui.T,Vj)))

@numba.jit
def grad_V(Vj, Yij, Ui, reg, eta):
    """
    Takes as input the column vector Vj (jth column of V^T), a training point Yij,
    Ui (the ith row of U), reg (the regularization parameter lambda),
    and eta (the learning rate).

    Returns the gradient of the regularized loss function with
    respect to Vj multiplied by eta.
    """
    # Gradient with respect to Vj
    return eta*(reg*Vj-Ui*(Yij-np.dot(Vj.T,Ui)))

@numba.jit
def get_err(U, V, Y, reg=0.0):
    """
    Takes as input a matrix Y of triples (i, j, Y_ij) where i is the index of a user,
    j is the index of a movie, and Y_ij is user i's rating of movie j and
    user/movie matrices U and V.

    Returns the mean regularized squared-error of predictions made by
    estimating Y_{ij} as the dot product of the ith row of U and the jth column of V^T.
    """
    err = 0
    # Compute squared error
    for i,j,Yij in Y:
        i=i-1
        j=j-1
        err = err + (Yij-np.dot(U[i],V[j]))**2
    # Return MSE + regularization
    return err/len(Y)/2+reg/2.0*(np.linalg.norm(U)**2+np.linalg.norm(V)**2)

@numba.jit
def train_model(M, N, K, eta, reg, Y, eps=0.0001, max_epochs=300):
    """
    Given a training data matrix Y containing rows (i, j, Y_ij)
    where Y_ij is user i's rating on movie j, learns an
    M x K matrix U and N x K matrix V such that rating Y_ij is approximated
    by (UV^T)_ij.

    Uses a learning rate of <eta> and regularization of <reg>. Stops after
    <max_epochs> epochs, or once the magnitude of the decrease in regularized
    MSE between epochs is smaller than a fraction <eps> of the decrease in
    MSE after the first epoch.

    Returns a tuple (U, V, err) consisting of U, V, and the unregularized MSE
    of the model.
    """
    # initial U and V to random floats from -0.5 to 0.5
    U = np.reshape(np.random.rand(M*K)-0.5,(M,K))
    V = np.reshape(np.random.rand(N*K)-0.5,(N,K))
    # Storage of U,V,err for each epoch
    data = []
    # For eps termination
    preverr = 100
    for epoch in range(0,max_epochs):
        # Shuffles the data before each epoch
        idx=np.random.permutation(np.arange(len(Y)))
        for i,j,Yij in Y[idx]:
            # Convert indices counting from 0
            i=i-1
            j=j-1
            Ui = U[i]
            Vj = V[j]
            # Randomly update U or V
            if np.random.randint(2):                
                U[i]=U[i]-grad_U(Ui, Yij, Vj, reg, eta)
            else:
                V[j]=V[j]-grad_V(Vj, Yij, Ui, reg, eta)
        # Compute the error
        err = get_err(U,V,Y)
        data.append([U,V,err])
        if epoch%5==0:
            print('SGD on epoch ',epoch,', error=',err)
        # Terminate if error reduction is less than eps
        if abs(err-preverr) < eps:
            print('EPS condition reached')
            break
        preverr = err
    # Final output
    data=np.array(data)
    return data[-1,0],data[-1,1],data[-1,2]

In [18]:
# Reader that reads tab-separated data
full = np.loadtxt("./data/data.txt")

# Hyperparameter optimization

In [ ]:
%%time
# Test out the latent model
Y_train = np.loadtxt('./data/train.txt').astype(int)
Y_test = np.loadtxt('./data/test.txt').astype(int)

M = max(max(Y_train[:,0]), max(Y_test[:,0])).astype(int) # users
N = max(max(Y_train[:,1]), max(Y_test[:,1])).astype(int) # movies
print("Factorizing with ", M, " users, ", N, " movies.")
Ks = [10,20,30,50,100]

reg = 0.1
eta = 0.03 # learning rate
U,V,err = train_model(M, N, Ks[1], eta, reg, Y_train)

Factorizing with  943  users,  1682  movies.
SGD on epoch  0 , error= 0.8293882299754082
SGD on epoch  5 , error= 0.4125036687598385
SGD on epoch  10 , error= 0.3714642873395004
SGD on epoch  15 , error= 0.35067387237350633
SGD on epoch  20 , error= 0.3348051133854553
SGD on epoch  25 , error= 0.319366314666794
SGD on epoch  30 , error= 0.3086820632406794
SGD on epoch  35 , error= 0.3028175514745843
SGD on epoch  40 , error= 0.2960228035720193
SGD on epoch  45 , error= 0.29206103756773055
SGD on epoch  50 , error= 0.2853965463047724
SGD on epoch  55 , error= 0.2868181514681604
SGD on epoch  60 , error= 0.2862695741037143
SGD on epoch  65 , error= 0.2839851960489585


# Training on Full Dataset, Getting Matrices

# SVD and Projection

In [ ]:
# Do SVD (singular value decomposition) on the matrices
A, Sigma, B = np.linalg.svd(V, full_matrices=False) # V = A @ np.diag(Sigma) @ B 
print(V.shape)
print(A.shape)
print(np.diag(Sigma).shape)
print(B.shape)

In [ ]:
# Take the first 2 rows(?) of A
A_red = A[:2, :]
print(A_red.shape)

# Multiply with V to reduce V from 20 dim to 2 dim
V_red = (V @ A_red.T)
print(V_red.shape)
print(V_red)

# Visualization

In [ ]:
# Scatter the 2 dimensions of the reduced V
plt.scatter(V_red[:, 0], V_red[:, 1])

# Top 10 most popular movies
for i in [50, 258, 100, 181, 294, 286, 288, 1, 300, 121]:
    plt.scatter(V_red[clf.trainset.to_inner_iid(str(i)), 0], V_red[clf.trainset.to_inner_iid(str(i)), 1], c='r')

In [ ]:
# TODO: Copy the code from the basic visualization that allowed us sort the movies so that we can subselect the relevant
# subsets of the movies.
for i in [50, 258, 100, 181, 294, 286, 288, 1, 300, 121]:
    plt.scatter(V_red[clf.trainset.to_inner_iid(str(i)), 0], V_red[clf.trainset.to_inner_iid(str(i)), 1])